#  Streamlit CardioRisk App
This notebook defines a simple interactive Streamlit app for predicting cardiovascular risk from clinical input.

In [3]:
import streamlit as st
import xgboost as xgb
import joblib
import pandas as pd
from sklearn.impute import SimpleImputer
import os


In [17]:
# Load model and feature order
MODEL_PATH = "../src/models/tuned_xgb_fhir.pkl"
FEATURE_PATH = "../src/models/fhir_feature_order.pkl"

booster = joblib.load(MODEL_PATH)
feature_order = joblib.load(FEATURE_PATH)
imputer = SimpleImputer(strategy="mean")


In [20]:
st.title(" CardioRisk Estimator")
st.markdown("Dự đoán nguy cơ mắc bệnh tim mạch trong 10 năm dựa trên chỉ số sinh học lâm sàng.")

# Input fields
input_data = {}
for feat in feature_order:
    if feat == "age":
        input_data[feat] = st.number_input("Tuổi", 18, 120, 55)
    elif "BP" in feat:
        input_data[feat] = st.number_input(feat, 50, 250, 120)
    elif "chol" in feat.lower():
        input_data[feat] = st.number_input(feat, 100, 400, 200)
    else:
        input_data[feat] = st.number_input(feat, 30, 300, 100)


2025-05-11 19:04:31.876 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-11 19:04:31.877 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-11 19:04:31.878 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-11 19:04:31.880 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-11 19:04:31.881 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-11 19:04:31.883 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-11 19:04:31.884 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-11 19:04:31.884 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [24]:
if st.button("🩺 Dự đoán nguy cơ"):
    df_input = pd.DataFrame([input_data])[feature_order]
    df_input = pd.DataFrame(imputer.fit_transform(df_input), columns=feature_order)
    dmat = xgb.DMatrix(df_input, feature_names=feature_order)
    pred = booster.predict(dmat)[0]
    st.metric(label="Xác suất nguy cơ bệnh tim", value=f"{pred:.2%}")


2025-05-11 19:07:57.524 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-11 19:07:57.526 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-11 19:07:57.528 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-11 19:07:57.529 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-11 19:07:57.530 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
